## Movielens 영화 추천 실습

In [1]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


### 1. 데이터 준비와 전처리

In [2]:
import os
import pandas as pd

rating_file_path='/aiffel/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
# 별점을 시청횟수로 생각함.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# MF model
data = pd.merge(ratings, movies)
data

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


### 2. 데이터 탐색
- ratings에 있는 유니크한 영화 개수

In [8]:
data['movie_id'].nunique()

3628

- ratings에 있는 유니크한 사용자 수

In [9]:
data['user_id'].nunique()

6039

- 가장 인기 있는 영화 30개(인기순)

In [10]:
popular_movie = data.groupby('title')['user_id'].count()
popular_movie.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [11]:
movies[movies['title'].str.contains("Toy Story")]

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
3045,3114,Toy Story 2 (1999),Animation|Children's|Comedy


In [12]:
movies[movies['title'].str.contains("Bug")]

,movie_id,title,genre
997,1010,"Love Bug, The (1969)",Children's|Comedy
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [13]:
movies[movies['title'].str.contains("Mummy")]

,movie_id,title,genre
2548,2617,"Mummy, The (1999)",Action|Adventure|Horror|Thriller
2564,2633,"Mummy, The (1932)",Horror|Romance
2565,2634,"Mummy, The (1959)",Horror
2566,2635,"Mummy's Curse, The (1944)",Horror
2567,2636,"Mummy's Ghost, The (1944)",Horror
2568,2637,"Mummy's Hand, The (1940)",Horror
2569,2638,"Mummy's Tomb, The (1942)",Horror


In [14]:
movies[movies['title'].str.contains("Back")]

,movie_id,title,genre
342,346,Backbeat (1993),Drama|Musical
1154,1170,Best of the Best 3: No Turning Back (1995),Action
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1250,1270,Back to the Future (1985),Comedy|Sci-Fi
1752,1817,No Looking Back (1998),Comedy|Drama|Romance
1794,1863,Major League: Back to the Minors (1998),Comedy
1942,2011,Back to the Future Part II (1989),Comedy|Sci-Fi
1943,2012,Back to the Future Part III (1990),Comedy|Sci-Fi|Western
2085,2154,How Stella Got Her Groove Back (1998),Drama|Romance
2311,2380,Police Academy 3: Back in Training (1986),Comedy


In [15]:
movies[movies['title'].str.contains("Rain Man")]

,movie_id,title,genre
1892,1961,Rain Man (1988),Drama


In [16]:
my_movies = ["Toy Story 2 (1999)", "Bug's Life, A (1998)", "Mummy, The (1999)", "Back to the Future (1985)", "Rain Man (1988)"]
df_movie = movies[movies['title'].isin(my_movies)]

my_counts = [5, 4, 4, 5, 4]
my_movie_list = pd.DataFrame({'user_id':['kim']*5, 'movie_id': df_movie['movie_id'], 'counts':my_counts})

if not data.isin({'user_id':['kim']})['user_id'].any():
    ratings = ratings.append(my_movie_list)            

ratings.tail(10)

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
1250,kim,1270,5,NaN
1892,kim,1961,4,NaN
2286,kim,2355,4,NaN
2548,kim,2617,5,NaN
3045,kim,3114,4,NaN


In [17]:
rating_ = ratings.copy()
del rating_['timestamp']
rating_

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1250,kim,1270,5
1892,kim,1961,4
2286,kim,2355,4
2548,kim,2617,5


In [18]:
data = pd.merge(rating_, movies)
data

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [19]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kim'])
print(movie_to_idx['Toy Story 2 (1999)'])

3497
50


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,title,genre
0,0,1193,5,0,Drama
1,1,1193,5,0,Drama
2,2,1193,4,0,Drama
3,3,1193,4,0,Drama
4,4,1193,5,0,Drama
...,...,...,...,...,...
836478,1621,3607,5,3623,Comedy|Drama|Western
836479,3481,3026,4,3624,Horror
836480,3481,690,3,3625,Romance
836481,4160,2909,4,3626,Documentary


### 4. CSR matrix

In [22]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5. als_model = AlternatingLeastSquares 모델

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지

regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지

use_gpu : GPU를 사용할 것인지

iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [25]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

### 6. 훈련된 모델이 예측한 선호도 파악

In [27]:
kim, Toy_Story_2 = user_to_idx['kim'], movie_to_idx['Toy Story 2 (1999)']
kim_vector, Toy_Story_2_vector = als_model.user_factors[kim], als_model.item_factors[Toy_Story_2]

In [28]:
kim_vector

array([ 1.0565939 , -0.04464793,  0.40099314, -0.5225664 , -0.99517703,
        0.2429053 , -0.5822836 , -0.37561464, -0.3870343 ,  0.24950774,
        0.2627538 ,  0.4799959 , -0.1264827 ,  0.39542118, -0.7129365 ,
        0.6086745 ,  0.2713171 , -0.22025229,  0.9669328 , -0.6001058 ,
        0.06486372, -0.24717638, -0.46218875, -0.8803364 , -0.43469328,
       -0.06417815,  0.8593464 , -0.00612804,  0.42889214,  0.54060644,
       -0.34655422, -0.18896367, -0.6457375 , -0.03749653,  0.27663803,
       -0.28263468, -1.0175484 ,  0.3533562 ,  0.47690973,  1.3579164 ,
       -1.2406965 , -0.42679298, -0.514296  , -0.38311145,  0.81684494,
       -0.6068054 , -0.11955594, -0.1798474 ,  0.10800018, -0.05377822,
        0.06599417,  0.30217463, -0.0302417 ,  1.207347  , -0.31122836,
       -0.15966147, -0.41670558,  1.1057633 , -0.62779635, -0.95389026,
        0.7031023 ,  0.3458818 , -0.8517804 , -0.31769237,  0.68985134,
        0.40652773, -0.7098107 , -0.13475375,  0.4520054 ,  0.42

In [29]:
Toy_Story_2_vector

array([ 0.01974711,  0.01631143,  0.01087052,  0.01495291, -0.02854905,
       -0.00965167, -0.00091934, -0.01123112,  0.01015097,  0.00208326,
        0.05107598,  0.02603824,  0.00204081,  0.00716872, -0.01517741,
        0.01076899,  0.04150581, -0.00933775,  0.0083933 , -0.01279023,
        0.01442979,  0.01374867, -0.02509711, -0.01219993, -0.01698417,
        0.01822978,  0.03100902, -0.01124313,  0.02931611,  0.03140271,
       -0.00643724,  0.01908354, -0.01429253,  0.02252007, -0.01471191,
       -0.00435515, -0.03654758, -0.00875476,  0.00106104,  0.03540701,
       -0.01735595,  0.01056255, -0.00496419, -0.00080487,  0.01681093,
       -0.0030865 , -0.02120237,  0.00801989,  0.01133874, -0.01269109,
       -0.00531364,  0.02894613, -0.0041486 ,  0.04425262, -0.00887534,
       -0.00075101, -0.01458459,  0.04519329, -0.03072909, -0.02435962,
       -0.02739926,  0.00886504, -0.01245857,  0.00529717,  0.01364753,
        0.0165964 ,  0.00051147, -0.02085187,  0.013886  ,  0.01

In [30]:
np.dot(kim_vector, Toy_Story_2_vector)

0.71509206

- 사용자와 아이템 벡터 내적수가 적절하게 형성된 것으로 보인다.

In [31]:
movies[movies['title'].str.contains("Mulan")]

,movie_id,title,genre
1838,1907,Mulan (1998),Animation|Children's


In [32]:
mulan = movie_to_idx['Mulan (1998)']
mulan_vector = als_model.item_factors[mulan]
np.dot(kim_vector, mulan_vector)

0.12118954

### 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [33]:
movies[movies['title'].str.contains("Life Is Beautiful")]

,movie_id,title,genre
2255,2324,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama


In [34]:
favorite_movie = 'Life Is Beautiful (La Vita è bella) (1997)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(450, 1.0000001),
 (122, 0.46118823),
 (98, 0.42904216),
 (424, 0.37354583),
 (473, 0.3676077),
 (757, 0.34641474),
 (53, 0.34543654),
 (732, 0.3406331),
 (154, 0.33739048),
 (481, 0.33464164),
 (1898, 0.33117723),
 (55, 0.32725868),
 (1471, 0.3255626),
 (445, 0.30891165),
 (90, 0.30500728)]

In [35]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Life Is Beautiful (La Vita è bella) (1997)',
 'Waking Ned Devine (1998)',
 'Misérables, Les (1998)',
 'Misérables, Les (1995)',
 'Hurricane, The (1999)',
 'Full Monty, The (1997)',
 'Shine (1996)',
 'Immortal Beloved (1994)',
 'As Good As It Gets (1997)',
 'Kolya (1996)',
 'Smoke Signals (1998)',
 'Shall We Dance? (Shall We Dansu?) (1996)',
 'Cinema Paradiso (1988)',
 'Red Violin, The (Le Violon rouge) (1998)',
 'Spanish Prisoner, The (1997)']

In [36]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [37]:
movies[movies['title'].str.contains("Twister")]

,movie_id,title,genre
727,736,Twister (1996),Action|Adventure|Romance|Thriller


In [38]:
get_similar_movie('Twister (1996)')

['Twister (1996)',
 'Con Air (1997)',
 'Cliffhanger (1993)',
 'Independence Day (ID4) (1996)',
 'Lethal Weapon 3 (1992)',
 'Saint, The (1997)',
 'G.I. Jane (1997)',
 "Dante's Peak (1997)",
 'Six Days Seven Nights (1998)',
 'River Wild, The (1994)']

### 8. 내가 가장 좋아할 만한 영화들을 추천받기

In [39]:
user = user_to_idx['kim']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(40, 0.46838504),
 (126, 0.268307),
 (243, 0.26733315),
 (851, 0.25840336),
 (1706, 0.23396991),
 (160, 0.20723268),
 (474, 0.20474663),
 (674, 0.19423136),
 (99, 0.18886086),
 (23, 0.18550482),
 (670, 0.18339072),
 (121, 0.18152611),
 (64, 0.17347522),
 (110, 0.17119145),
 (107, 0.16608),
 (707, 0.16578346),
 (332, 0.1656675),
 (322, 0.1643946),
 (48, 0.15972619),
 (19, 0.15756443)]

In [40]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Toy Story (1995)',
 'Shakespeare in Love (1998)',
 'Ghostbusters (1984)',
 'Iron Giant, The (1999)',
 'Stuart Little (1999)',
 'Forrest Gump (1994)',
 'Chicken Run (2000)',
 'Back to the Future Part II (1989)',
 'American Beauty (1999)',
 "Schindler's List (1993)",
 'Galaxy Quest (1999)',
 'Silence of the Lambs, The (1991)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Groundhog Day (1993)',
 'Jurassic Park (1993)',
 'Sting, The (1973)',
 'Moonstruck (1987)',
 'Babe (1995)',
 'Saving Private Ryan (1998)',
 'Big (1988)']

In [41]:
chicken_run = movie_to_idx['Chicken Run (2000)']
explain = als_model.explain(user, csr_data, itemid=chicken_run)

In [42]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[("Bug's Life, A (1998)", 0.08182786603508273),
 ('Toy Story 2 (1999)', 0.06415033030607094),
 ('Rain Man (1988)', 0.03597723247734952),
 ('Mummy, The (1999)', 0.01227180362207367),
 ('Back to the Future (1985)', 0.0004252229850842349)]

<ol>
<ul> 1. <strong>프로젝트 설명</strong>
    <li> Movielens 사이트의 영화 데이터를 바탕으로 좋아하는 영화를 관심 작품을 추천해주는 프로그램을 만드는 프로젝트를 진행하였다. 사용자의 취향에 잘 맞는 영화를 추천하는 것이 목표이다.</li>
</ul>
<ul> 2. <strong>시도한 것들</strong>
    <li> 학습에 사용하는 하이퍼 파라미터를 바꾸어보고 다양한 영화를 활용해보았다.</li>
</ul>
<ul> 3. <strong>알게된 점</strong>
    <li> 추천시스템에서는 사용자와 영화의 벡터 내적값이 1에 가까우면 좋지만, 어떤 값이 나와야 모델이 적절한지에 대한 객관적인 지표는 없다.</li>
</ul>
<ul> 4. <strong>어려웠던 점</strong>
    <li> 영화의 제목에 개봉연도가 포함되어 있고 일부는 The가 뒤쪽에 있어서 전처리를 진행하려고 하였다. 그러나 같은 제목인데 다른 해에 개봉한 영화가 있는 것을 확인하고 전처리를 진행하지 않았다. 데이터셋에 대한 이해가 충분하지 않아 어려움을 겪었다.</li>
</ul>
<ul> 5. <strong>아쉬운 점</strong>
    <li> 추천 결과로 나온 영화 중에 아는 영화가 많지 않아 제대로 추천하고 있는지 확인하기가 쉽지 않았다. 한국 영화를 포함해서 최신 영화를 위주로 추천 프로그램을 만들어보면 좋을 것 같다.</li>
</ul>
</ol>